# 🛡️ Complete Robustness Analysis

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Time:</b> 25 minutes<br>
<b>Dataset:</b> Wine (sklearn)<br>
<b>Prerequisite:</b> 01_tests_introduction.ipynb
</div>

## 🎯 Objectives
- ✅ Understand what robustness is in ML
- ✅ Run complete robustness analysis
- ✅ Interpret perturbation methods
- ✅ Analyze sensitive features
- ✅ Generate professional HTML report

## 📚 What is Robustness?

### Definition
**Robustness** is the model's ability to maintain good performance even when input data suffers small perturbations.

### Why it matters?
1. **Real-world data is imperfect**
   - Measurement errors
   - Sensor noise
   - Manual data entry

2. **Security against adversarial attacks**
   - Intentional feature manipulation
   - Attempts to fool the model

3. **Production reliability**
   - Model must be stable
   - Consistent predictions

## 1️⃣ Setup and Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from deepbridge import DBDataset, Experiment

# Configure visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load data
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target

print(f"📊 Dataset: {df.shape}")
print(f"🍷 Classes: {wine.target_names}")

## 2️⃣ Train Model

In [ ]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

baseline_acc = clf.score(X_test, y_test)
print(f"✅ Model trained!")
print(f"📊 Baseline accuracy: {baseline_acc:.3f}")

## 3️⃣ Create Experiment

In [ ]:
dataset = DBDataset(
    data=df,
    target_column='target',
    model=clf,
    test_size=0.2,
    random_state=42,
    dataset_name='Wine Classification'
)

exp = Experiment(
    dataset=dataset,
    experiment_type='multiclass_classification',
    random_state=42
)

print("✅ Experiment created!")

## 4️⃣ Quick Test

First, let's run a quick test:

In [ ]:
print("🧪 Running quick robustness test...\n")

rob_quick = exp.run_test('robustness', config_name='quick')

print("\n✅ Quick test completed!")
print(f"\n📊 Robustness Score: {rob_quick.get('robustness_score', 'N/A')}")

## 5️⃣ Complete Test (full)

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #ff9800;">
<b>⏱️ Note:</b> The complete test may take a few minutes!
</div>

In [ ]:
print("🔬 Running COMPLETE robustness analysis...\n")

rob_full = exp.run_test('robustness', config_name='full')

print("\n✅ Complete analysis finished!")

## 6️⃣ Perturbation Methods

DeepBridge tests robustness using different methods:

In [ ]:
print("📊 TESTED PERTURBATION METHODS:\n" + "="*60 + "\n")

# List methods and results
if 'perturbation_methods' in rob_full:
    for method, result in rob_full['perturbation_methods'].items():
        print(f"\n🔹 {method.upper()}")
        print(f"   Description: {result.get('description', 'N/A')}")
        print(f"   Score: {result.get('score', 'N/A')}")
        print(f"   Degradation: {result.get('degradation', 'N/A')}")
else:
    print("\n🔹 GAUSSIAN NOISE")
    print("   Adds Gaussian noise to numerical features")
    print("   Simulates: Measurement errors, sensor noise\n")
    
    print("🔹 DROPOUT")
    print("   Randomly removes feature values")
    print("   Simulates: Missing data, sensor failures\n")
    
    print("🔹 SCALING")
    print("   Multiplies features by random factors")
    print("   Simulates: Changes in measurement scale\n")
    
    print("🔹 ADVERSARIAL")
    print("   Targeted perturbations to fool the model")
    print("   Simulates: Intentional adversarial attacks")

## 7️⃣ Interpret Robustness Score

### Score Scale
- **0.95 - 1.00**: 🟢 Excellent - Very robust model
- **0.85 - 0.95**: 🟡 Good - Acceptable robustness
- **0.70 - 0.85**: 🟠 Moderate - Needs attention
- **< 0.70**: 🔴 Weak - Fragile model

In [ ]:
score = rob_full.get('robustness_score', 0)

print(f"\n📊 ROBUSTNESS SCORE: {score:.3f}\n")

if score >= 0.95:
    print("🟢 EXCELLENT - Very robust model!")
    print("   ✅ Ready for production")
elif score >= 0.85:
    print("🟡 GOOD - Acceptable robustness")
    print("   ✅ Can go to production with monitoring")
elif score >= 0.70:
    print("🟠 MODERATE - Needs attention")
    print("   ⚠️  Consider improvements before production")
else:
    print("🔴 WEAK - Fragile model")
    print("   ❌ NOT recommended for production")

## 8️⃣ Sensitive Features

Analyze which features are most sensitive to perturbations:

In [ ]:
# Example: If available in result
if 'feature_sensitivity' in rob_full:
    sensitivity = rob_full['feature_sensitivity']
    
    # Top 5 most sensitive features
    print("\n🔍 TOP 5 MOST SENSITIVE FEATURES:\n" + "="*60)
    for i, (feature, sens_score) in enumerate(sorted(sensitivity.items(), 
                                                      key=lambda x: x[1], 
                                                      reverse=True)[:5], 1):
        print(f"\n{i}. {feature}")
        print(f"   Sensitivity: {sens_score:.3f}")
        print(f"   ⚠️  Perturbations in this feature greatly affect the model")
else:
    print("\n💡 Feature sensitivity not available in this configuration")
    print("   Use config='full' for complete analysis")

## 9️⃣ Visualize Performance Degradation

In [ ]:
# Create degradation visualization
if 'degradation_curve' in rob_full:
    degradation = rob_full['degradation_curve']
    
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Degradation curve
    plt.subplot(1, 2, 1)
    plt.plot(degradation['noise_levels'], degradation['accuracy'], 
             marker='o', linewidth=2, markersize=8)
    plt.axhline(y=baseline_acc, color='r', linestyle='--', 
                label=f'Baseline ({baseline_acc:.3f})', alpha=0.7)
    plt.xlabel('Perturbation Level', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Performance Degradation', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Method comparison
    plt.subplot(1, 2, 2)
    methods = list(rob_full.get('perturbation_methods', {}).keys())
    scores = [rob_full['perturbation_methods'][m].get('score', 0) for m in methods]
    
    if methods:
        plt.barh(methods, scores, color='skyblue', edgecolor='navy', alpha=0.7)
        plt.xlabel('Robustness Score', fontsize=12)
        plt.title('Score by Perturbation Method', fontsize=14, fontweight='bold')
        plt.xlim(0, 1)
        plt.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("📊 Degradation data not available")
    print("   Run with config='full' for complete visualizations")

## 🔟 Generate HTML Report

Generate professional report for documentation:

In [ ]:
# Generate HTML report
output_path = '/tmp/robustness_report.html'

if hasattr(rob_full, 'save_html'):
    rob_full.save_html(output_path)
    print(f"✅ HTML report generated!")
    print(f"📁 Location: {output_path}")
    print(f"\n💡 Open this file in your browser to view the complete report")
else:
    print("💡 To generate HTML reports, use:")
    print("   exp.save_report('robustness', output_path)")

## 1️⃣1️⃣ How to Improve Robustness?

### Techniques to Increase Robustness:

#### 1. **Data Augmentation**
```python
# Add noise during training
X_train_noisy = X_train + np.random.normal(0, 0.1, X_train.shape)
```

#### 2. **Ensemble Methods**
```python
# Use multiple models (we're already using RandomForest)
from sklearn.ensemble import VotingClassifier
```

#### 3. **Robust Feature Engineering**
- Remove very sensitive features
- Create aggregated features
- Proper normalization

#### 4. **Regularization**
```python
# Increase regularization (L1, L2, dropout)
```

#### 5. **Adversarial Training**
```python
# Train with adversarial examples
```

## 🎉 Conclusion

### What you learned:
- ✅ **Robustness Concept** - Resistance to perturbations
- ✅ **Perturbation Methods** - Gaussian, Dropout, Scaling, Adversarial
- ✅ **Interpret Scores** - Scale from 0 to 1
- ✅ **Sensitive Features** - Identify vulnerabilities
- ✅ **Reports** - Professional documentation
- ✅ **Improvement Techniques** - How to increase robustness

### Robustness Checklist for Production:
- [ ] ✅ Robustness Score > 0.85
- [ ] ✅ Sensitive features identified and documented
- [ ] ✅ Acceptable degradation in all methods
- [ ] ✅ HTML report generated for audit
- [ ] ✅ Production monitoring plan

### Next Steps:
- 📘 `03_uncertainty.ipynb` - Quantify uncertainty in predictions
- 📘 `04_resilience_drift.ipynb` - Detect temporal drift
- 📘 `06_model_comparison.ipynb` - Compare robustness between models

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>💡 Insight:</b> A model with high accuracy but low robustness can fail in production!
</div>